# YOLOv5 - Prediccion


En este notebook, se desarrolla la predicción del modelo YOLOv5 para la detección de insectos en fotografías de plantas, destinado a generar una red de polinizadores. 

## Paquetes

In [ ]:
import torch
from yolov5 import YOLOv5
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Revisión del modelo sin entrenar


Cargamos el modelo yolov5m, entrenado con el dataset COCO, sin entrenar, para ver como se comporta con imágenes de nuestro dataset.

In [3]:
model = YOLOv5("yolov5m.pt", device="cpu")

# Recorro las imágenes de test
detecciones = []
images = os.listdir('dataset/images/test')
for img in images:
    # Obtengo la predicción
    results = model.predict("dataset/images/test/" + img)
    # Procesar los resultados
    detection = results.pandas().xyxy[0]
    # Tomar la detección con mayor confianza
    detection = detection[detection["confidence"] == detection["confidence"].max()]
    # Tomo solo name
    try:
        detection = detection["name"].values[0]
    except:
        detection = 'no_detection'
    # Guardo la detección
    detecciones.append(detection)

# Genero el dataframe con las imagenes y las detecciones
df = pd.DataFrame({'imagen': images, 'deteccion': detecciones})
# Reviso
display(df.head(5))


YOLOv5  2023-12-12 Python-3.8.10 torch-1.13.1+cpu CPU


Fusing layers... 
YOLOv5m summary: 369 layers, 21190557 parameters, 0 gradients
Adding AutoShape... 


,imagen,deteccion
0,00068ee5e4632cf05be16306f00aa51b.jpg,no_detection
1,0065808ec8a40acb4dbaf1a855045a1b.jpg,no_detection
2,00720ef5141a225e95a973c785e5292b.jpg,orange
3,00ad5ae0ef6fe82cad95e54e8f478857.jpg,bird
4,00ad5ae0ef6fe82cad95e54e8f478857_reflejada1.jpg,bird


Realizamos un conteo de las predicciones.

In [4]:
# Conteo de detecciones
df['deteccion'].value_counts()

no_detection    853
bird            322
banana          109
dining table     98
apple            67
potted plant     25
microwave        19
orange           19
broccoli         18
chair            16
bear             16
person           15
carrot           14
teddy bear        7
train             7
tv                5
vase              4
giraffe           4
bench             3
bowl              3
dog               2
sports ball       2
frisbee           2
umbrella          2
cat               1
surfboard         1
laptop            1
kite              1
cake              1
knife             1
sheep             1
Name: deteccion, dtype: int64

# Modelo entrenado


Usamos el modelo entrenado con el dataset de plantas e insectos, cuyos pesos se encuentran en el archivo `yolov5l_v01.pt`.

In [11]:
model = YOLOv5("yolov5l_v01.pt", device="cpu")  # usa "cuda" si tienes GPU

# Recorro las imágenes de validación
detecciones = []
images = os.listdir('dataset/images/test')
for img in images:
    # Obtengo la predicción
    results = model.predict("dataset/images/test/" + img)
    # Procesar los resultados
    detection = results.pandas().xyxy[0]
    # Tomar la detección con mayor confianza
    detection = detection[detection["confidence"] == detection["confidence"].max()]
    # Tomo solo name
    try:
        detection = detection["name"].values[0]
    except:
        detection = 'no_detection'
    # Guardo la detección
    detecciones.append(detection)

# Genero el dataframe con las imagenes y las detecciones
df = pd.DataFrame({'imagen': images, 'deteccion': detecciones})
# Reviso
display(df.head(5))


YOLOv5  2023-12-12 Python-3.8.10 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 368 layers, 46170604 parameters, 0 gradients, 108.3 GFLOPs
Adding AutoShape... 


,imagen,deteccion
0,00068ee5e4632cf05be16306f00aa51b.jpg,no_detection
1,0065808ec8a40acb4dbaf1a855045a1b.jpg,no_detection
2,00720ef5141a225e95a973c785e5292b.jpg,no_detection
3,00ad5ae0ef6fe82cad95e54e8f478857.jpg,Wasp
4,00ad5ae0ef6fe82cad95e54e8f478857_reflejada1.jpg,Wasp


Realizamos un conteo de las predicciones.

In [12]:
# Conteo de detecciones
df['deteccion'].value_counts()

no_detection    531
Bee             329
Diptera         247
Lepidoptera     185
Coleoptera      152
Others          105
Wasp             77
Hoverfly         13
Name: deteccion, dtype: int64

Unimos con la información de las imágenes.

In [13]:
# Leo el dataset
data = pd.read_csv('dataset/info_imagenes_cat.csv')
# Diccionario de categorias 
cat = { 0: 'Bee', 1: 'Lepidoptera', 2: 'Diptera', 3: 'Coleoptera', 4: 'Others', 5: 'Wasp', 6: 'Hoverfly'}
# Cambio las categorias por números
data['categoría'] = data['categoría'].map(cat)
# Merge
data = data.merge(df, on='imagen')
# Reviso
display(data.head(5))

,imagen,xmin,ymin,xmax,ymax,categoría,deteccion
0,00068ee5e4632cf05be16306f00aa51b.jpg,327,424,660,881,Diptera,no_detection
1,0065808ec8a40acb4dbaf1a855045a1b.jpg,766,138,993,462,Bee,no_detection
2,00720ef5141a225e95a973c785e5292b.jpg,338,291,978,827,Bee,no_detection
3,00ad5ae0ef6fe82cad95e54e8f478857.jpg,226,162,1054,706,Wasp,Wasp
4,01204afe7194a9a98cd57bb629877716.jpg,554,312,1095,649,Bee,Bee


Realizamos las matrices de confusión, tanto absoluta como normalizada.

In [14]:
# Realizo una matriz de confusión entre las categorías y las detecciones
display(pd.crosstab(data['categoría'], data['deteccion']))
# porcentual
display(pd.crosstab(data['categoría'], data['deteccion'], normalize='index'))

deteccion,Bee,Coleoptera,Diptera,Hoverfly,Lepidoptera,Others,Wasp,no_detection
categoría,,,,,,,,
Bee,324,2,13,5,1,28,8,187
Coleoptera,4,201,22,0,3,6,3,127
Diptera,4,2,235,1,0,13,0,116
Hoverfly,0,0,0,7,0,0,0,3
Lepidoptera,0,0,0,0,208,0,0,55
Others,3,4,3,0,2,150,2,78
Wasp,3,2,0,0,0,2,69,30


deteccion,Bee,Coleoptera,Diptera,Hoverfly,Lepidoptera,Others,Wasp,no_detection
categoría,,,,,,,,
Bee,0.570423,0.003521,0.022887,0.008803,0.001761,0.049296,0.014085,0.329225
Coleoptera,0.010929,0.549180,0.060109,0.000000,0.008197,0.016393,0.008197,0.346995
Diptera,0.010782,0.005391,0.633423,0.002695,0.000000,0.035040,0.000000,0.312668
Hoverfly,0.000000,0.000000,0.000000,0.700000,0.000000,0.000000,0.000000,0.300000
Lepidoptera,0.000000,0.000000,0.000000,0.000000,0.790875,0.000000,0.000000,0.209125
Others,0.012397,0.016529,0.012397,0.000000,0.008264,0.619835,0.008264,0.322314
Wasp,0.028302,0.018868,0.000000,0.000000,0.000000,0.018868,0.650943,0.283019


Realizamos el mapa de calor.

In [28]:
tabla = pd.crosstab(data['categoría'], data['deteccion'], normalize='index')
sns.heatmap(tabla, cmap="crest", annot=True, fmt=".2f")